it will be useful to have a cheap-and-cheerful way of inputting simple tunes.

C2C2G2G2AB<C>AG2 F2E2D2C2
    


In [1]:
octave = 4 # global var tracking 'reference octave'


def midi_note(letter):
    '''returns midi note as a numerical code given letter and reference octave'''
    note_offset = {'C': 0, 'D': 2, 'E': 4, 'F': 5, 'G': 7, 'A': 9, 'B': 11}
    return octave * 12 + note_offset[letter]

def next_note(line):
    letters = set('ABCDEFG')
    global octave
    ch = line.pop(0)
    if ch =='.':
        return -1, 0 # end of tune
    if ch in set('<>'): #shift reference octave up or down
        if ch == '<':
            octave += 1
        else: octave += -1
        return -1, 1
    if ch in letters:
        note = midi_note(ch)
        if line and line[0] in set('+-'): # sharp or flat
            porm = line.pop(0)
            if porm == '+':
                note += 1
            else: note += -1
        if line and line[0] in set('123456789abcdefg'): # relative duration
            dur = line.pop(0)
            duration = 1 + list('123456789abcdefg').index(dur)
        else: duration = 1
        return note, duration

def string_to_midi(tune_string):
    tune = list(tune_string.replace(" ", ""))
    score = []
    while tune:
        note, duration = next_note(tune)
        if note > 0:
            score.append((note, duration))
    return score

score1 = string_to_midi("F2F2<C2C2DEFDC4 ")
score2 = string_to_midi(">B-2B-2A2A2G2G2F4")



In [2]:
import sounddevice as sd
from math import pi
import numpy as np
import matplotlib.pyplot as plt

In [14]:

def pitch(n):
    return 440 * 2**((n-57)/12)
    
class Player:

    def __init__(self, stream):
        self.stream = stream
        self.rate = stream.samplerate
        self.one_sec = np.linspace(0, 440*2*pi, int(self.rate))
        self.range = 2**30

    def play_note(self, f, d):
        "Play a note with injected (invented) attack and decay."
        s_len = int(self.rate*d)
        t = np.linspace(0, d, s_len)
        ad_len = int(s_len*0.05)
        ad_mult = np.array(s_len * [1.0])
        ad_mult[:ad_len] = np.linspace(0, 1, ad_len)
        ad_mult[-ad_len:] = np.linspace(1, 0, ad_len)
        samples = (self.range*np.cos(2*pi*f*t)).astype('int32')
        self.stream.write(samples)
        return samples

with sd.OutputStream(channels=1, dtype='int32') as stream:
    p = Player(stream)
    for (note, duration) in score1:
        s = p.play_note(pitch(note), 0.125 * duration)
    for (note, duration) in score2:
        s = p.play_note(pitch(note), 0.125 * duration)

In [19]:
with sd.OutputStream(channels=1, dtype='int32') as stream:
    p = Player(stream)
    for (note, duration) in score2:
        s = p.play_note(pitch(note), 0.125 * duration)

In [17]:
score2

[(58, 2), (58, 2), (57, 2), (57, 2), (55, 2), (55, 2), (53, 4)]